In [8]:
from __future__ import division
from __future__ import print_function

import os
import glob
import time
import random
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from utils import load_data, accuracy
from models import GAT, SpGAT, mGAT

In [9]:
"""
# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False, help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False, help='Validate during training pass.')
parser.add_argument('--sparse', action='store_true', default=False, help='GAT with sparse version or not.')
parser.add_argument('--seed', type=int, default=72, help='Random seed.')
parser.add_argument('--epochs', type=int, default=10000, help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.005, help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4, help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=8, help='Number of hidden units.')
parser.add_argument('--nb_heads', type=int, default=8, help='Number of head attentions.')
parser.add_argument('--dropout', type=float, default=0.6, help='Dropout rate (1 - keep probability).')
parser.add_argument('--alpha', type=float, default=0.2, help='Alpha for the leaky_relu.')
parser.add_argument('--patience', type=int, default=100, help='Patience')

args = parser.parse_args()
"""

class ARGS :
    def __init__(self):
        self.no_cuda = False # Disables CUDA training.
        self.fastmode = False # Validate during training pass.
        self.sparse = False # GAT with sparse version or not.
        self.seed = 72 # Random seed.
        self.epochs = 10000 # Number of epochs to train.
        self.lr = 0.005 # Initial learning rate.
        self.weight_decay = 5e-4 # Weight decay (L2 loss on parameters).
        self.hidden = 8 # Number of hidden units.
        self.nb_heads = 8 # Number of head attentions.
        self.dropout = 0.6 # Dropout rate (1 - keep probability).
        self.alpha = 0.2 # Alpha for the leaky_relu.
        self.patience = 100 # Patience
        self.cuda = not self.no_cuda and torch.cuda.is_available()
args = ARGS()

In [10]:
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...
data loaded : 2708 nodes, 1433 features, 7 labels


In [11]:
# Model and optimizer
if args.sparse:
    model = SpGAT(nfeat=features.shape[1], 
                nhid=args.hidden, 
                nclass=int(labels.max()) + 1, 
                dropout=args.dropout, 
                nheads=args.nb_heads, 
                alpha=args.alpha)
else:
    model = mGAT(nfeat=features.shape[1], 
                nhid=args.hidden, 
                nclass=int(labels.max()) + 1, 
                dropout=args.dropout, 
                nheads=args.nb_heads, 
                alpha=args.alpha)
optimizer = optim.Adam(model.parameters(), 
                       lr=args.lr, 
                       weight_decay=args.weight_decay)

if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

features, adj, labels = Variable(features), Variable(adj), Variable(labels)


def train(epoch, bad_counter):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.data.item()),
          'acc_train: {:.4f}'.format(acc_train.data.item()),
          'loss_val: {:.4f}'.format(loss_val.data.item()),
          'acc_val: {:.4f}'.format(acc_val.data.item()),
          'time: {:.4f}s'.format(time.time() - t),
          'bad_counter: {:04d}'.format(bad_counter))

    return loss_val.data.item(), acc_val.data.item()


def compute_test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data.item()),
          "accuracy= {:.4f}".format(acc_test.data.item()))

In [12]:


# Train model
t_total = time.time()
loss_values = []
acc_values = []
bad_counter = 0
best = args.epochs + 1
best_epoch = 0
for epoch in range(args.epochs):
    loss_val, acc_val = train(epoch, bad_counter)
    loss_values.append(loss_val)
    acc_values.append(acc_val)


    torch.save(model.state_dict(), '{}.pkl'.format(epoch))
    if loss_values[-1] < best:
        best = loss_values[-1]
        best_epoch = epoch
        bad_counter = 0
    else:
        bad_counter += 1

    if bad_counter == args.patience:
        break

    files = glob.glob('*.pkl')
    for file in files:
        epoch_nb = int(file.split('.')[0])
        if epoch_nb < best_epoch:
            os.remove(file)

files = glob.glob('*.pkl')
for file in files:
    epoch_nb = int(file.split('.')[0])
    if epoch_nb > best_epoch:
        os.remove(file)

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0001 loss_train: 1.9458 acc_train: 0.1714 loss_val: 1.9408 acc_val: 0.3233 time: 28.1020s bad_counter: 0000
Epoch: 0002 loss_train: 1.9370 acc_train: 0.2714 loss_val: 1.9361 acc_val: 0.3700 time: 28.5508s bad_counter: 0000
Epoch: 0003 loss_train: 1.9348 acc_train: 0.3286 loss_val: 1.9308 acc_val: 0.3933 time: 28.5727s bad_counter: 0000
Epoch: 0004 loss_train: 1.9300 acc_train: 0.3643 loss_val: 1.9255 acc_val: 0.3900 time: 28.9468s bad_counter: 0000


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')


def plt_hist(loss_values, acc_values):
    
    acr_val = acc_values
    
    loss_val = loss_values
    x = range(1, len(loss_val) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    #plt.plot(x, accu, 'b', label='Training accu')
    plt.plot(x, acr_val, 'r', label='Validation accu')
    plt.title('validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    #plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, loss_val, 'r', label='Validation loss')
    plt.title('validation loss')
    plt.legend()
    
plt_hist(loss_values, acc_values)



In [ ]:
# Restore best model
print('Loading {}th epoch'.format(best_epoch))
model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))

# Testing
compute_test()